### Loading libraries and files

In [1]:
import pandas as pd
import re
import pickle
from urllib.request import Request, urlopen
import bs4 as bs
import lxml
import json
import random
import pickle
import ast
import random
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
import ftfy
import langdetect
pd.set_option("display.max_columns", None)

In [2]:
df = pd.read_csv("../Data/filtered_df_muckrack_with_twitter8april2021.csv")
for i in range(len(df)):
    journos = json.loads(df.iloc[i].journalist)
    if len(journos) < 3:
        df.loc[i, "journalist"] = 'delete'
df = df[df.journalist != 'delete'].reset_index()

### Filter by ranking and followers

In [3]:
def get_alexa_ranking(url):
    try:
        xml = urlopen('http://data.alexa.com/data?cli=10&dat=s&url={}'.format(url)).read() 
        text = int(re.search(r'<POPULARITY[^>]*TEXT="(\d+)"', str(xml)).groups()[0])
    except:
        text = "Cannot find"
    return text

In [4]:
df['alexa_ranking'] = df.apply(lambda x: get_alexa_ranking(x['cleaned_outlet_url']), axis=1)

In [5]:
df = df[df.alexa_ranking != "Cannot find"].reset_index()

In [6]:
df = df[df.t_followers.notnull()]

In [7]:
def clean_followers(num):
    return int(num.replace(',', ''))

In [8]:
# df['clean_t_followers'] = df.apply(lambda x: clean_followers(x['t_followers']), axis=1)

In [9]:
# df.info()

In [10]:
# df.clean_t_followers.mean(), df.clean_t_followers.median()

In [11]:
# df.alexa_ranking.mean(), df.alexa_ranking.median()

In [12]:
# select_df = df[df.clean_t_followers >= df.clean_t_followers.median()]
# select_df = select_df[select_df.alexa_ranking <= df.alexa_ranking.median()]
# select_df = select_df.drop(columns=['level_0', 'index']).reset_index()

In [14]:
# select_df.to_pickle('../Data/select_df.pkl')
with open('../Data/select_df.pkl', 'rb') as f:
    select_df = pickle.load(f)

### Get unique journalists

In [15]:
unique_journos = pd.DataFrame(columns = ['name', 'profile_url', 'beats'])
for i in range(len(select_df)):
    j = json.loads(select_df.iloc[i].journalist)
    for k in j:
        unique_journos = unique_journos.append({'name': k['name'], 'profile_url': k['profile_url'], 
                                                'beats': '-'},  ignore_index = True)

In [16]:
unique_journos = unique_journos.drop_duplicates().reset_index()[['name', 'profile_url', 'beats']]

In [17]:
unique_journos

,name,profile_url,beats
0,"Celli, Rita",https://muckrack.com/cbcontariotoday,-
1,"Draaisma, Muriel",https://muckrack.com/murieldraaisma,-
2,"Reddekopp, Lorenda",https://muckrack.com/cbclorenda,-
3,"Zhou, Lu",https://muckrack.com/lu-zhou-1,-
4,"Ashry, Dalia",https://muckrack.com/dalia-ashry,-
...,...,...,...
31094,"Trimble, Grady",https://muckrack.com/grady-trimble,-
31095,"Varney, Stuart",https://muckrack.com/stuart-varney,-
31096,"Webster, Ashley",https://muckrack.com/ashwebsterfbn,-
31097,"Whittemore, Chelsea",https://muckrack.com/chelsea-whittemore,-


### Pick 1000 journalists and check for validity of beats

In [18]:
random_rows = [random.randint(0, len(unique_journos) - 1) for i in range(1000)]

In [19]:
topics = ['Arts and Entertainment', 'Beauty', 'Business and Finance', 'Crime and Justice', 'Education', 
                'Energy', 'Environment', 'Fashion', 'Food and Dining', 'Health', 'Media', 'Military', 
                'Opinion and Editorial',  'Politics', 'Real Estate', 'Religion', 'Science', 'Sports', 
                'Technology', 'Transportation', 'Travel', 'Weather', 'World']

In [20]:
for i in random_rows:
    try:
        url = unique_journos.iloc[i].profile_url
        req = Request(url, headers = {'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
        soup = bs.BeautifulSoup(sauce,'lxml')
        beats = soup.find(class_ = 'person-details-beats').find_all('a')
        beats = [i.text for i in beats]
        beats = [i for i in beats if i in topics]
    except:
        beats = '-'
    unique_journos.iloc[i].beats = beats

In [21]:
unique_journos = unique_journos[unique_journos["beats"] != '-']

In [22]:
unique_journos = unique_journos[unique_journos["beats"].str.len() != 0]

In [23]:
unique_journos = unique_journos.reset_index()[['name', 'profile_url', 'beats']]

In [24]:
unique_journos

,name,profile_url,beats
0,"Bridge, Sarah",https://muckrack.com/sarah-bridge,[Opinion and Editorial]
1,"Collins, Leah",https://muckrack.com/leah-collins,[Arts and Entertainment]
2,"Hopton, Alice",https://muckrack.com/alice-hopton,[Arts and Entertainment]
3,"Kapelos, Vassy",https://muckrack.com/vassykapelos,[Politics]
4,"McCreadie, Danielle",https://muckrack.com/danielle-mccreadie,"[Education, Environment, Politics]"
...,...,...,...
688,"Ramos, Dino-Ray",https://muckrack.com/dino-ray-ramos,[Arts and Entertainment]
689,"Schlager, Brandon",https://muckrack.com/brandon-schlager,[Sports]
690,"Dane, Chris",https://muckrack.com/chris-dane,[Business and Finance]
691,"Genovese, Daniella",https://muckrack.com/daniella-genovese,[Business and Finance]


In [25]:
for i in range(len(unique_journos)):
    print(unique_journos.iloc[i].beats)

['Opinion and Editorial']
['Arts and Entertainment']
['Arts and Entertainment']
['Politics']
['Education', 'Environment', 'Politics']
['Politics']
['Arts and Entertainment', 'Media']
['Politics']
['Politics']
['Sports']
['Arts and Entertainment']
['Arts and Entertainment']
['Technology']
['Environment', 'Weather']
['Technology']
['Crime and Justice', 'Politics', 'Technology']
['Business and Finance', 'Education']
['Sports']
['World']
['Sports']
['Sports']
['World']
['Arts and Entertainment']
['Business and Finance', 'Politics']
['Politics']
['Environment', 'Transportation']
['Media', 'Politics', 'World']
['Sports']
['Arts and Entertainment']
['Environment']
['Business and Finance']
['Politics']
['Science']
['Arts and Entertainment']
['Sports']
['Sports']
['Sports']
['Opinion and Editorial']
['Health']
['Politics']
['Politics', 'World']
['Business and Finance']
['Health']
['Arts and Entertainment']
['Opinion and Editorial']
['Arts and Entertainment']
['Arts and Entertainment']
['Crime a

### Fetch articles for all journalists

In [26]:
nltk.download('stopwords')
nltk.download('wordnet')
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [27]:
all_articles = pd.DataFrame(columns = ['name', 'url', 'title', 'description', 'content', 'beats', 'predicted'])
all_articles

,name,url,title,description,content,beats,predicted


In [28]:
for i in range(len(unique_journos)):
    
    print(i)
    try:
    
        name = unique_journos.iloc[i]['name']
        muckrack = unique_journos.iloc[i]['profile_url'] + '/articles'

        req = Request(muckrack, headers = {'User-Agent': 'Mozilla/5.0'})
        sauce = urlopen(req).read()
        soup = bs.BeautifulSoup(sauce,'lxml')

        beats = unique_journos.iloc[i]['beats']

        articles = soup.find_all(class_ = "news-story")

        for article in articles:
            title = article.find(class_ = 'news-story-title').text
            description = article.find(class_ = 'news-story-body').text.split('—')[1].strip()
            if langdetect.detect(description) == 'en':
                # content = clean(title * 4 + description * 2)
                content = '-'

                # print(name, muckrack, title, description, content, beat)
                all_articles = all_articles.append({'name': name, 'url': muckrack, 'title': title, 'description': description, 
                                                'content': content, 'beats': beats, 'predicted': '-'}, ignore_index = True)
    
    except:
        pass

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

### Select 5000 random articles and clean them

In [29]:
# drop articles with duplicate titles, and shuffle them before picking random articles
all_articles = all_articles.drop_duplicates(subset = ['title']).sample(frac = 1).reset_index(drop = True)
all_articles

,name,url,title,description,content,beats,predicted
0,"Peters, Chris",https://muckrack.com/chris-peters/articles,\nHockey Sense Roundup: Sullivan to coach USA ...,This has been one of the busiest weeks when it...,-,[Sports],-
1,"Rao, Sonia",https://muckrack.com/sonia-rao/articles,\nDaBaby’s homophobic remarks draw a week’s wo...,Rapper DaBaby has faced a week of intense crit...,-,[Arts and Entertainment],-
2,"Johnson, Miles",https://muckrack.com/miles-johnson-1/articles,\nSilvio Berlusconi: Italy’s great survivor pl...,Standing in the sun outside Milan’s San Raffae...,-,[Business and Finance],-
3,"Samuelson, Robert",https://muckrack.com/person-2354966/articles,\nOpinion | The stock market’s ‘fear gauge’ hi...,What are stocks telling us? What explains thei...,-,"[Business and Finance, Opinion and Editorial, ...",-
4,"Tilden, Imogen",https://muckrack.com/imogen-tilden/articles,\nClassical highlights: concerts and operas to...,Royal Opera House Christmas ConcertConducted b...,-,[Arts and Entertainment],-
...,...,...,...,...,...,...,...
10782,"Dellenger, Ross",https://muckrack.com/rossdellenger/articles,\nD.J. Uiagalelei: Clemson QB to profit from N...,Through the first six weeks of the NIL Era in ...,-,[Sports],-
10783,"Steele, Anne",https://muckrack.com/annemariesteele/articles,\nBritney Spears Moves to Remove Father From C...,Britney Spears’s new lawyer moved to have her ...,-,"[Arts and Entertainment, Business and Finance]",-
10784,"Bagchi, Rob",https://muckrack.com/rob-bagchi/articles,\nlive score and latest updates\n,And welcome to coverage of day two of The Hund...,-,[Sports],-
10785,"Milani, Kate",https://muckrack.com/kate-milani/articles,\nCapital Journal Daybreak: What’s at Stake in...,"HIGHLIGHTS In Georgia House Race, Parties Batt...",-,"[Business and Finance, Politics]",-


In [30]:
random_articles_indexes = [random.randint(0, len(all_articles) - 1) for i in range(5000)]
len(random_articles_indexes)

5000

In [31]:
random_articles = all_articles.loc[random_articles_indexes].reset_index(drop = True)

In [32]:
random_articles

,name,url,title,description,content,beats,predicted
0,"Walsh, Lara",https://muckrack.com/lara-walsh/articles,\nThis Glass-Encased Airbnb Surrounded By Volc...,TravelIt also includes a hot air balloon ride ...,-,"[Food and Dining, Travel]",-
1,"Stango, Nicholas",https://muckrack.com/nicholas-stango/articles,\nOnce Upon A Time In The West Is the Best Wes...,The opening is iconic. In eerie silence three ...,-,[Media],-
2,"Steinberg, Brian",https://muckrack.com/bristei/articles,"\nWeekend ‘GMA’ Anchor Dan B. Harris, to Leave...","Dan B. Harris, an anchor who has served multip...",-,"[Arts and Entertainment, Media]",-
3,"Siemaszko, Corky",https://muckrack.com/corky-siemaszko/articles,\nJapan shelters Belarusian sprinter who sough...,Krystsina Tsimanouskaya made a dash for freedo...,-,[Technology],-
4,"Sharma, Samrat",https://muckrack.com/_samratsharma/articles,\nNagpur outnumbers Mumbai's Covid cases; Vida...,"India reported 23,285 new Covid-19 cases on Th...",-,[Business and Finance],-
...,...,...,...,...,...,...,...
4995,"Wiederer, Dan",https://muckrack.com/dan-wiederer/articles,\nFrom Teven Jenkins to Akiem Hicks: The top 1...,Chicago Bears defensive lineman Akiem Hicks wa...,-,[Sports],-
4996,"Kavanagh, Sean",https://muckrack.com/sean-kavanagh/articles,\nCritics attack proposed changes to bylaws go...,A report suggesting changes to how the City of...,-,[Politics],-
4997,"Schroeder, Rob",https://muckrack.com/mktwrobs/articles,\nJill Biden to have procedure on foot after s...,First lady Jill Biden will on Thursday undergo...,-,[Politics],-
4998,"Conlan, Tara",https://muckrack.com/tara-conlan-1/articles,\nI'm a Celebrity ... so socially distance and...,Presenters Ant McPartlin and Declan Donnelly w...,-,[Politics],-


In [34]:
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}

regex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
# Cleaning the text sentences so that punctuation marks, stop words &amp; digits are removed
# Words are lemmatized according to their POS tags
def clean(doc):
    doc = ftfy.fix_text(doc)
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = ""
    for word in punc_free.split():
        tag = nltk.pos_tag([word])[0][1][0].upper()
        normalized = normalized + " " + str(lemma.lemmatize(word, tag_dict.get(tag, wordnet.NOUN)))
    processed = re.sub(r"\d+","", normalized)
    y = processed.split()
    return ' '.join(y)

def remove_emojis(text):
    try:
        return regex_pattern.sub(r'',text)
    except:
        return np.nan

In [44]:
for i in range(len(random_articles)):
#     print(i)
    title = random_articles.iloc[i].title
    description = random_articles.iloc[i].description
    content = title + " " + title + " " + title + " " + title + " " + description + " " + description
    content = clean(content)

    try:
        content = remove_emojis(content)
    except:
        pass
    
#     print(content)
    
    try:
        lang = langdetect.detect(content)
        if lang != 'en':
            content = np.nan
    except Exception as e:
#         print(e)
        content = np.nan
        
#     print(content, i)
        
    random_articles.iloc[i].content = content

In [45]:
random_articles.content

0       glassencased airbnb surround volcano cost glas...
1       upon time west best western watch upon time we...
2       weekend gma anchor dan b harris leave abc news...
3       japan shelter belarusian sprinter sought flee ...
4       nagpur outnumbers mumbai covid case vidarbha g...
                              ...                        
4995    teven jenkins akiem hick top injury absence co...
4996    critic attack propose change bylaw govern winn...
4997    jill biden procedure foot step object white ho...
4998    im celebrity socially distance wash hand im ce...
4999    southwest airline automates job recruiting tas...
Name: content, Length: 5000, dtype: object

### Predict tiers using IAB classifier

In [49]:
iab_mod = pickle.load(open('../Models/IAB_classifier.p','rb'))
iab_vec = pickle.load(open('../Models/IAB_vectorizer.p','rb'))
iab_bin = pickle.load(open('../Models/IAB_binarizer.p','rb'))
tiers = pd.read_csv("../Data/iab content taxonomy tiers.csv", engine = "python")
tier1_tags = tiers["Tier 1"].unique()

In [50]:
def iab_taxonomy_v2(text):
    text = iab_vec.transform([text])
    res = list(iab_bin.inverse_transform(iab_mod.predict(text))[0])
    res = [i for i in res if i in tier1_tags]
    if '-' in res:
        res.remove('-')
    return list(set(res).intersection(set(tier1_tags))) if res else '-'

In [53]:
random_articles = random_articles[~random_articles.content.isna()].reset_index(drop = True)
random_articles

,name,url,title,description,content,beats,predicted
0,"Walsh, Lara",https://muckrack.com/lara-walsh/articles,\nThis Glass-Encased Airbnb Surrounded By Volc...,TravelIt also includes a hot air balloon ride ...,glassencased airbnb surround volcano cost glas...,"[Food and Dining, Travel]",-
1,"Stango, Nicholas",https://muckrack.com/nicholas-stango/articles,\nOnce Upon A Time In The West Is the Best Wes...,The opening is iconic. In eerie silence three ...,upon time west best western watch upon time we...,[Media],-
2,"Steinberg, Brian",https://muckrack.com/bristei/articles,"\nWeekend ‘GMA’ Anchor Dan B. Harris, to Leave...","Dan B. Harris, an anchor who has served multip...",weekend gma anchor dan b harris leave abc news...,"[Arts and Entertainment, Media]",-
3,"Siemaszko, Corky",https://muckrack.com/corky-siemaszko/articles,\nJapan shelters Belarusian sprinter who sough...,Krystsina Tsimanouskaya made a dash for freedo...,japan shelter belarusian sprinter sought flee ...,[Technology],-
4,"Sharma, Samrat",https://muckrack.com/_samratsharma/articles,\nNagpur outnumbers Mumbai's Covid cases; Vida...,"India reported 23,285 new Covid-19 cases on Th...",nagpur outnumbers mumbai covid case vidarbha g...,[Business and Finance],-
...,...,...,...,...,...,...,...
4966,"Wiederer, Dan",https://muckrack.com/dan-wiederer/articles,\nFrom Teven Jenkins to Akiem Hicks: The top 1...,Chicago Bears defensive lineman Akiem Hicks wa...,teven jenkins akiem hick top injury absence co...,[Sports],-
4967,"Kavanagh, Sean",https://muckrack.com/sean-kavanagh/articles,\nCritics attack proposed changes to bylaws go...,A report suggesting changes to how the City of...,critic attack propose change bylaw govern winn...,[Politics],-
4968,"Schroeder, Rob",https://muckrack.com/mktwrobs/articles,\nJill Biden to have procedure on foot after s...,First lady Jill Biden will on Thursday undergo...,jill biden procedure foot step object white ho...,[Politics],-
4969,"Conlan, Tara",https://muckrack.com/tara-conlan-1/articles,\nI'm a Celebrity ... so socially distance and...,Presenters Ant McPartlin and Declan Donnelly w...,im celebrity socially distance wash hand im ce...,[Politics],-


In [54]:
random_articles['predicted'] = random_articles.apply(lambda x: iab_taxonomy_v2(x['content']), axis = 1)
random_articles

,name,url,title,description,content,beats,predicted
0,"Walsh, Lara",https://muckrack.com/lara-walsh/articles,\nThis Glass-Encased Airbnb Surrounded By Volc...,TravelIt also includes a hot air balloon ride ...,glassencased airbnb surround volcano cost glas...,"[Food and Dining, Travel]",-
1,"Stango, Nicholas",https://muckrack.com/nicholas-stango/articles,\nOnce Upon A Time In The West Is the Best Wes...,The opening is iconic. In eerie silence three ...,upon time west best western watch upon time we...,[Media],-
2,"Steinberg, Brian",https://muckrack.com/bristei/articles,"\nWeekend ‘GMA’ Anchor Dan B. Harris, to Leave...","Dan B. Harris, an anchor who has served multip...",weekend gma anchor dan b harris leave abc news...,"[Arts and Entertainment, Media]",-
3,"Siemaszko, Corky",https://muckrack.com/corky-siemaszko/articles,\nJapan shelters Belarusian sprinter who sough...,Krystsina Tsimanouskaya made a dash for freedo...,japan shelter belarusian sprinter sought flee ...,[Technology],[Content Language]
4,"Sharma, Samrat",https://muckrack.com/_samratsharma/articles,\nNagpur outnumbers Mumbai's Covid cases; Vida...,"India reported 23,285 new Covid-19 cases on Th...",nagpur outnumbers mumbai covid case vidarbha g...,[Business and Finance],[Content Source Geo]
...,...,...,...,...,...,...,...
4966,"Wiederer, Dan",https://muckrack.com/dan-wiederer/articles,\nFrom Teven Jenkins to Akiem Hicks: The top 1...,Chicago Bears defensive lineman Akiem Hicks wa...,teven jenkins akiem hick top injury absence co...,[Sports],[Medical Health]
4967,"Kavanagh, Sean",https://muckrack.com/sean-kavanagh/articles,\nCritics attack proposed changes to bylaws go...,A report suggesting changes to how the City of...,critic attack propose change bylaw govern winn...,[Politics],[Content Language]
4968,"Schroeder, Rob",https://muckrack.com/mktwrobs/articles,\nJill Biden to have procedure on foot after s...,First lady Jill Biden will on Thursday undergo...,jill biden procedure foot step object white ho...,[Politics],[Content Source Geo]
4969,"Conlan, Tara",https://muckrack.com/tara-conlan-1/articles,\nI'm a Celebrity ... so socially distance and...,Presenters Ant McPartlin and Declan Donnelly w...,im celebrity socially distance wash hand im ce...,[Politics],[Pop Culture]


In [55]:
random_articles.to_csv('../Data/double_randomized_results.csv')

### Mapping tiers and interpreting results

In [56]:
random_articles = pd.read_csv('../Data/double_randomized_results.csv')

In [57]:
random_articles = random_articles.drop(columns= 'Unnamed: 0')

In [58]:
random_articles

,name,url,title,description,content,beats,predicted
0,"Walsh, Lara",https://muckrack.com/lara-walsh/articles,\nThis Glass-Encased Airbnb Surrounded By Volc...,TravelIt also includes a hot air balloon ride ...,glassencased airbnb surround volcano cost glas...,"['Food and Dining', 'Travel']",-
1,"Stango, Nicholas",https://muckrack.com/nicholas-stango/articles,\nOnce Upon A Time In The West Is the Best Wes...,The opening is iconic. In eerie silence three ...,upon time west best western watch upon time we...,['Media'],-
2,"Steinberg, Brian",https://muckrack.com/bristei/articles,"\nWeekend ‘GMA’ Anchor Dan B. Harris, to Leave...","Dan B. Harris, an anchor who has served multip...",weekend gma anchor dan b harris leave abc news...,"['Arts and Entertainment', 'Media']",-
3,"Siemaszko, Corky",https://muckrack.com/corky-siemaszko/articles,\nJapan shelters Belarusian sprinter who sough...,Krystsina Tsimanouskaya made a dash for freedo...,japan shelter belarusian sprinter sought flee ...,['Technology'],['Content Language']
4,"Sharma, Samrat",https://muckrack.com/_samratsharma/articles,\nNagpur outnumbers Mumbai's Covid cases; Vida...,"India reported 23,285 new Covid-19 cases on Th...",nagpur outnumbers mumbai covid case vidarbha g...,['Business and Finance'],['Content Source Geo']
...,...,...,...,...,...,...,...
4966,"Wiederer, Dan",https://muckrack.com/dan-wiederer/articles,\nFrom Teven Jenkins to Akiem Hicks: The top 1...,Chicago Bears defensive lineman Akiem Hicks wa...,teven jenkins akiem hick top injury absence co...,['Sports'],['Medical Health']
4967,"Kavanagh, Sean",https://muckrack.com/sean-kavanagh/articles,\nCritics attack proposed changes to bylaws go...,A report suggesting changes to how the City of...,critic attack propose change bylaw govern winn...,['Politics'],['Content Language']
4968,"Schroeder, Rob",https://muckrack.com/mktwrobs/articles,\nJill Biden to have procedure on foot after s...,First lady Jill Biden will on Thursday undergo...,jill biden procedure foot step object white ho...,['Politics'],['Content Source Geo']
4969,"Conlan, Tara",https://muckrack.com/tara-conlan-1/articles,\nI'm a Celebrity ... so socially distance and...,Presenters Ant McPartlin and Declan Donnelly w...,im celebrity socially distance wash hand im ce...,['Politics'],['Pop Culture']


In [59]:
random_articles.predicted.value_counts()

-                                 1588
['Content Source Geo']             816
['Business and Finance']           439
['Sports']                         413
['Personal Finance']               168
['Medical Health']                 151
['Events and Attractions']         134
['Technology & Computing']         133
['Content Language']               128
['Music and Audio']                113
['Style & Fashion']                 81
['Automotive']                      76
['Education']                       69
['Video Gaming']                    60
['Family and Relationships']        56
['Travel']                          49
['Hobbies & Interests']             44
['News and Politics']               43
['Fine Art']                        40
['Television']                      40
['Books and Literature']            38
['Movies']                          37
['Food & Drink']                    33
['Pop Culture']                     33
['Healthy Living']                  32
['Careers']              

In [60]:
random_articles = random_articles[random_articles.predicted != "-"]
random_articles = random_articles[random_articles.predicted != "['Content Source Geo']"]
random_articles = random_articles[random_articles.predicted != "['Content Language']"]
random_articles = random_articles[random_articles.predicted != "['Content Channel']"]
random_articles = random_articles[random_articles.predicted != "['Content Type']"]
random_articles = random_articles[random_articles.predicted != "['Content Source']"]
random_articles = random_articles[random_articles.predicted != "['Content Media Format']"].reset_index(drop = True)
random_articles

,name,url,title,description,content,beats,predicted
0,"Settimi, Christina",https://muckrack.com/christina-settimi/articles,\nThe Most Valuable Esports Companies 2020\n,It’s been a tough year for esports. Soaring va...,valuable esports company valuable esports comp...,"['Business and Finance', 'Sports']",['Video Gaming']
1,"Pardes, Arielle",https://muckrack.com/ariellepardes/articles,\n9 ways newsrooms can incorporate more audio ...,The ability to hear stories is essential for v...,way newsroom incorporate audio work way newsro...,['Technology'],['Hobbies & Interests']
2,"Morgan, Abigail Daisy",https://muckrack.com/abigail-daisy-morgan/arti...,\nThe secret to start-up survival: Entrepreneu...,Incredible inventions are being created and th...,secret startup survival entrepreneur tell tale...,"['Business and Finance', 'World']",['Business and Finance']
3,"Winkleman, Claudia",https://muckrack.com/claudia-winkleman/articles,\nClaudia Winkleman: 'My female friendships mi...,"Claudia Winkleman, 49, is a television and rad...",claudia winkleman my female friendship might t...,['Arts and Entertainment'],['Music and Audio']
4,"Saenger, Peter",https://muckrack.com/peter-saenger/articles,\nThe Star-Maker of Paris\n,Henri de Toulouse-Lautrec (1864-1901) was the ...,starmaker paris starmaker paris starmaker pari...,['Arts and Entertainment'],['Music and Audio']
...,...,...,...,...,...,...,...
2398,"Caranicas, Peter",https://muckrack.com/peter-caranicas/articles,\n‘Mission: Impossible - Fallout’ and ‘Roma’ W...,'Chernobyl' and 'Tom Clancy’s Jack Ryan' take ...,mission impossible fallout rom win lmgi award ...,"['Arts and Entertainment', 'Media']",['Movies']
2399,"Stango, Nicholas",https://muckrack.com/nicholas-stango/articles,\nEscape the Cold By Watching 12 Angry Men\n,It's freezing. Literally everywhere. So let's ...,escape cold watch angry men escape cold watch ...,['Media'],['Style & Fashion']
2400,"Kayhart, Kevin",https://muckrack.com/kevin-kayhart/articles,\nKelly Ripa and Mark Consuelos' son thinks 'i...,Kelly Ripa and husband Mark Consuelos aren't s...,kelly ripa mark consuelos son think it fine sh...,"['Arts and Entertainment', 'Sports']",['Pop Culture']
2401,"Wiederer, Dan",https://muckrack.com/dan-wiederer/articles,\nFrom Teven Jenkins to Akiem Hicks: The top 1...,Chicago Bears defensive lineman Akiem Hicks wa...,teven jenkins akiem hick top injury absence co...,['Sports'],['Medical Health']


In [61]:
random_articles.predicted.value_counts()

['Business and Finance']          439
['Sports']                        413
['Personal Finance']              168
['Medical Health']                151
['Events and Attractions']        134
['Technology & Computing']        133
['Music and Audio']               113
['Style & Fashion']                81
['Automotive']                     76
['Education']                      69
['Video Gaming']                   60
['Family and Relationships']       56
['Travel']                         49
['Hobbies & Interests']            44
['News and Politics']              43
['Television']                     40
['Fine Art']                       40
['Books and Literature']           38
['Movies']                         37
['Food & Drink']                   33
['Pop Culture']                    33
['Healthy Living']                 32
['Careers']                        24
['Real Estate']                    19
['Pets']                           17
['Shopping']                       16
['Home & Gar

In [62]:
mapping = {'Arts and Entertainment':['Events and Attractions', 'Fine Art', 'Pop Culture', 'Books and Literature', 'Movies', 'Music and Audio', 'Television'],
'Beauty':['Style & Fashion', 'Shopping'],
'Business and Finance':['Brand Suitability and Risk', 'Business and Finance', 'Personal Finance'],
'Crime and Justice':['News and Politics'],
'Education':['Education', 'Careers'],
'Energy':['Automotive', 'Technology & Computing'],
'Environment':['Home & Garden', 'Healthy Living'],
'Fashion':['Style & Fashion', 'Shopping'],
'Food and Dining':['Food & Drink'],
'Health':['Healthy Living', 'Medical Health'],
'Media':['News and Politics', 'Movies', 'Music and Audio', 'Pop Culture', 'Video Gaming', 'Technology & Computing', 'Television'],
'Military':['News and Politics'],
'Opinion and Editorial':['Business and Finance', 'Medical Health', 'Personal Finance', 'News and Politics', 'Technology & Computing', 'Sports', 'Travel'],
'Politics':['Business and Finance', 'Personal Finance', 'Events and Attractions', 'News and Politics', 'Sensitive Topics'],
'Real Estate':['Real Estate'],
'Religion':['Religion & Spirituality'],
'Science':['Medical Health', 'Science'],
'Sports':['Events and Attractions', 'Sports', 'Video Gaming'],
'Technology':['Music and Audio', 'Technology & Computing', 'Television', 'Video Gaming'],
'Transportation':['Travel'],
'Travel':['Automotive', 'Travel', 'Events and Attractions'],
'Weather':['Science', 'Home & Garden'],
'World':['Business and Finance', 'Medical Health', 'Personal Finance', 'News and Politics', 'Technology & Computing', 'Sports', 'Travel']}

In [63]:
mapped_tiers = []
mapped_correctly = []

for i in range(len(random_articles)):
    beats = ast.literal_eval(random_articles.iloc[i].beats)
    # print(beats)
    temp = []
    for beat in beats:
        j = mapping[beat.strip()]
        temp = temp + j
    # print(temp)
#     print(beats, '\n', temp, '\n', random_articles.iloc[i].predicted)
    correct = True if set(ast.literal_eval(random_articles.iloc[i].predicted)).intersection(set(temp)) else False
#     print(correct)
    mapped_tiers.append(temp)
    mapped_correctly.append(correct)
#     print()

In [64]:
random_articles['mapped_tiers'] = mapped_tiers
random_articles['mapped_correctly'] = mapped_correctly
mapped_tiers

[['Brand Suitability and Risk',
  'Business and Finance',
  'Personal Finance',
  'Events and Attractions',
  'Sports',
  'Video Gaming'],
 ['Music and Audio', 'Technology & Computing', 'Television', 'Video Gaming'],
 ['Brand Suitability and Risk',
  'Business and Finance',
  'Personal Finance',
  'Business and Finance',
  'Medical Health',
  'Personal Finance',
  'News and Politics',
  'Technology & Computing',
  'Sports',
  'Travel'],
 ['Events and Attractions',
  'Fine Art',
  'Pop Culture',
  'Books and Literature',
  'Movies',
  'Music and Audio',
  'Television'],
 ['Events and Attractions',
  'Fine Art',
  'Pop Culture',
  'Books and Literature',
  'Movies',
  'Music and Audio',
  'Television'],
 ['Automotive', 'Technology & Computing'],
 ['Business and Finance',
  'Personal Finance',
  'Events and Attractions',
  'News and Politics',
  'Sensitive Topics',
  'Automotive',
  'Travel',
  'Events and Attractions'],
 ['Events and Attractions', 'Sports', 'Video Gaming'],
 ['Brand Suit

In [65]:
random_articles['mapped_correctly'].value_counts()

True     1405
False     998
Name: mapped_correctly, dtype: int64

In [66]:
random_articles[random_articles.mapped_correctly == True]

,name,url,title,description,content,beats,predicted,mapped_tiers,mapped_correctly
0,"Settimi, Christina",https://muckrack.com/christina-settimi/articles,\nThe Most Valuable Esports Companies 2020\n,It’s been a tough year for esports. Soaring va...,valuable esports company valuable esports comp...,"['Business and Finance', 'Sports']",['Video Gaming'],"[Brand Suitability and Risk, Business and Fina...",True
2,"Morgan, Abigail Daisy",https://muckrack.com/abigail-daisy-morgan/arti...,\nThe secret to start-up survival: Entrepreneu...,Incredible inventions are being created and th...,secret startup survival entrepreneur tell tale...,"['Business and Finance', 'World']",['Business and Finance'],"[Brand Suitability and Risk, Business and Fina...",True
3,"Winkleman, Claudia",https://muckrack.com/claudia-winkleman/articles,\nClaudia Winkleman: 'My female friendships mi...,"Claudia Winkleman, 49, is a television and rad...",claudia winkleman my female friendship might t...,['Arts and Entertainment'],['Music and Audio'],"[Events and Attractions, Fine Art, Pop Culture...",True
4,"Saenger, Peter",https://muckrack.com/peter-saenger/articles,\nThe Star-Maker of Paris\n,Henri de Toulouse-Lautrec (1864-1901) was the ...,starmaker paris starmaker paris starmaker pari...,['Arts and Entertainment'],['Music and Audio'],"[Events and Attractions, Fine Art, Pop Culture...",True
6,"Curran, David",https://muckrack.com/david-curran/articles,\nCar crashes into SF home after police pursuit\n,"Nov. 19, 2020Updated: Nov. 19, 2020 1:16 p.m. ...",car crash sf home police pursuit car crash sf ...,"['Politics', 'Travel']",['Automotive'],"[Business and Finance, Personal Finance, Event...",True
...,...,...,...,...,...,...,...,...,...
2394,"Zorthian, Julia",https://muckrack.com/julia-zorthian/articles,\nBest Books of Fall: September 2018\n,"Fall is arriving, and with it, a new crop of m...",best book fall september best book fall septem...,"['Arts and Entertainment', 'Real Estate']",['Books and Literature'],"[Events and Attractions, Fine Art, Pop Culture...",True
2395,"Karmin, Craig",https://muckrack.com/craigkarmin/articles,\nMarriott’s Next CEO Likely to Come From Insi...,Marriott International Inc. suffered another b...,marriotts next ceo likely come inside marriott...,"['Business and Finance', 'Real Estate']",['Business and Finance'],"[Brand Suitability and Risk, Business and Fina...",True
2396,"Patterson, Chip",https://muckrack.com/chip-patterson/articles,\nCollege football rankings: Alabama tops Clem...,The defending national champions will start wh...,college football ranking alabama top clemson p...,['Sports'],['Sports'],"[Events and Attractions, Sports, Video Gaming]",True
2398,"Caranicas, Peter",https://muckrack.com/peter-caranicas/articles,\n‘Mission: Impossible - Fallout’ and ‘Roma’ W...,'Chernobyl' and 'Tom Clancy’s Jack Ryan' take ...,mission impossible fallout rom win lmgi award ...,"['Arts and Entertainment', 'Media']",['Movies'],"[Events and Attractions, Fine Art, Pop Culture...",True


In [68]:
random_articles.to_csv('../Data/randomized_final.csv')